In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
from data_extraction import drought_data

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
input_file = "drought_data/us-droughts.csv"
droughts_df=pd.read_csv(input_file)
droughts_df = droughts_df.drop(droughts_df[['validStart', 'validEnd', 'domStatisticFormatID']], axis=1)
droughts_df.releaseDate = pd.to_datetime(droughts_df.releaseDate, format='%Y-%m-%d')

In [3]:
droughts_df["D0"]=droughts_df["D0"]-droughts_df["D1"]
droughts_df["D1"]=droughts_df["D1"]-droughts_df["D2"]
droughts_df["D2"]=droughts_df["D2"]-droughts_df["D3"]
droughts_df["D3"]=droughts_df["D3"]-droughts_df["D4"]
#adding a drought level based on https://www.kaggle.com/balagpdy/heatmap-animation-us-drought-map
### Calculate drought level (when NONE is 100% => 0, if D4 is 100% => 5, and linearly between 0 and 5)
droughts_df["Level"]=(droughts_df["D0"]+droughts_df["D1"]*2+droughts_df["D2"]*3+droughts_df["D3"]*4+droughts_df["D4"]*5)/100
droughts_df

,releaseDate,FIPS,county,state,NONE,D0,D1,D2,D3,D4,Level
0,2000-11-07,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,0.0000
1,2000-10-31,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,0.0000
2,2000-10-24,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,0.0000
3,2000-10-17,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,0.0000
4,2000-10-10,2013,Aleutians East Borough,AK,100.00,0.00,0.0,0.0,0.0,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
2829496,2015-12-08,56045,Weston County,WY,29.45,70.55,0.0,0.0,0.0,0.0,0.7055
2829497,2015-12-01,56045,Weston County,WY,29.45,70.55,0.0,0.0,0.0,0.0,0.7055
2829498,2015-11-24,56045,Weston County,WY,29.45,70.55,0.0,0.0,0.0,0.0,0.7055
2829499,2015-11-17,56045,Weston County,WY,29.45,70.55,0.0,0.0,0.0,0.0,0.7055


In [4]:
input_file = "drought_data/county_info_2016.csv"
#To read this file properly, due to the format it is recommended that you copy its contents to a new excel file.
#There seems to be a format issue.
county_df=pd.read_csv(input_file)
county_df

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 2: invalid continuation byte

In [ ]:
droughtwcount_df=pd.merge(droughts_df,county_df,how='inner',left_on="FIPS",right_on="GEOID",sort=False)

In [ ]:
droughtwcount_dfm = droughtwcount_df.set_index('releaseDate').groupby(['FIPS']).resample('M').mean()

In [ ]:
#droughtwcount_dfm = droughtwcount_df.set_index('releaseDate').groupby(['FIPS']).resample('M').mean()
droughtwcount_dfm

In [ ]:

#droughtwcount_df.rename(columns={"INTPTLONG                                                                                                               ":"INTPTLONG"})
droughtwcount_dfm2=droughtwcount_dfm[["FIPS","NONE","D0","D1","D2","D3","D4","Level","ALAND_SQMI","AWATER_SQMI","INTPTLAT","INTPTLONG                                                                                                               "]]
droughtwcount_dfm2.rename(columns={"INTPTLONG                                                                                                               ":"INTPTLONG"})


In [ ]:
droughtwcount_dfm2=droughtwcount_dfm2.dropna()
droughtwcount_dfm2=droughtwcount_dfm2.reset_index(level=1)
droughtwcount_dfm2=droughtwcount_dfm2.drop("FIPS",axis=1)
droughtwcount_dfm2.reset_index(inplace=True)
droughtwcount_dfm2
#dr_m.reset_index(level=1)

In [ ]:
test=drought_data()
test